# Interface locating

basic tasks

1) See if I have located all the particles in the gel by running paintByLocations

2) Generate raw interface stack with gel locations blacked out.

3) Relocate with hashes etc...but only the interface stack

In [2]:
# preamble
import numpy as np
import numba

import sys
sys.path.extend(['/Users/zsolt/Colloid_git/TractionRheoscopy'])
from data_analysis import static as da
from data_analysis import grid_track, rotation
from particleLocating import paintByLocations as pbl
import pandas as pd
import yaml
from importlib import reload
import seaborn as sns

In [8]:
# load gel locations 
gelPos = da.loadExperiment(fNameDict={'gelPos': 'tfrGel10212018A_shearRun10292018f_gel_stitched_pandas.h5'})['gelPos']

In [12]:
gelPos_t0 = gelPos.xs(0,level='frame')
gelPos_t0

,"x (um, imageStack)","y (um, imageStack)","z (um, imageStack)",x_std,y_std,z_std
particle,,,,,,
0,11.061014,17.360880,4.273772,0.161318,0.154410,0.141765
1,28.329169,44.502572,5.451974,0.185608,0.193725,0.132625
2,4.535116,30.755712,5.776501,0.166145,0.159693,0.137178
3,29.681689,13.350886,5.578862,0.177129,0.204540,0.176749
4,6.492181,34.793476,6.180907,0.156805,0.164460,0.143846
...,...,...,...,...,...,...
3165,205.852231,215.517305,13.016421,0.284404,0.282810,0.299678
3166,223.868141,229.844683,23.776225,0.111443,0.116534,0.088651
3167,197.364440,201.825359,24.349485,0.155706,0.155696,0.123837


In [15]:
gelPos_t0['x (px, z150_z225 Interface)'] = gelPos_t0['x (um, imageStack)']/0.115
gelPos_t0['y (px, z150_z225 Interface)'] = gelPos_t0['y (um, imageStack)']/0.115
gelPos_t0['z (px, z150_z225 Interface)'] = gelPos_t0['z (um, imageStack)']/0.15 - 150

/Users/zsolt/.conda/envs/tractionRheoscopy/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/zsolt/.conda/envs/tractionRheoscopy/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/zsolt/.conda/envs/tractionRheoscopy/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

In [18]:
gelPos_t0

,"x (um, imageStack)","y (um, imageStack)","z (um, imageStack)",x_std,y_std,z_std,"x (px, z150_z225 Interface)","y (px, z150_z225 Interface)","z (px, z150_z225 Interface)"
particle,,,,,,,,,
0,11.061014,17.360880,4.273772,0.161318,0.154410,0.141765,96.182727,150.964174,-121.508189
1,28.329169,44.502572,5.451974,0.185608,0.193725,0.132625,246.340601,386.978884,-113.653504
2,4.535116,30.755712,5.776501,0.166145,0.159693,0.137178,39.435791,267.440972,-111.489996
3,29.681689,13.350886,5.578862,0.177129,0.204540,0.176749,258.101645,116.094659,-112.807586
4,6.492181,34.793476,6.180907,0.156805,0.164460,0.143846,56.453745,302.551968,-108.793956
...,...,...,...,...,...,...,...,...,...
3165,205.852231,215.517305,13.016421,0.284404,0.282810,0.299678,1790.019398,1874.063522,-63.223863
3166,223.868141,229.844683,23.776225,0.111443,0.116534,0.088651,1946.679483,1998.649421,8.508166
3167,197.364440,201.825359,24.349485,0.155706,0.155696,0.123837,1716.212521,1755.003126,12.329902


In [48]:
idx = gelPos_t0[gelPos_t0['z (px, z150_z225 Interface)'] > 0 ].index
col = ['{} (px, z150_z225 Interface)'.format(x) for x in ['z','y','x']]
fPath = '/Volumes/TFR/tfrGel10212018A_shearRun10292018f/interfaceLocating'
fName = 'gelPos.xyz'
da.df2xyz(gelPos_t0.loc[idx][col],fPath,fName)

'/Volumes/TFR/tfrGel10212018A_shearRun10292018f/interfaceLocating/gelPos.xyz'

In [26]:
gelPos_t0.loc[idx][col].to_csv(fPath+'/{}'.format('gelPos.csv'), sep = ',')

In [51]:
from particleLocating import flatField 
reload(pbl)
overlay = pbl.locationOverlay(fPath +'/{}'.format('gelPos.csv'), fPath+'/{}'.format('t0000_z150_z225.tif'),locatingprogram='trackpy', locColumns = col)

In [52]:
from particleLocating import pyFiji
print(pyFiji.send2Fiji([overlay.glyphImage,overlay.inputPadded],wdir=fPath+'/'))

Images saved to tif and copied to system clipboard.
open("/Volumes/TFR/tfrGel10212018A_shearRun10292018f/interfaceLocating/tmp_0.tif");
open("/Volumes/TFR/tfrGel10212018A_shearRun10292018f/interfaceLocating/tmp_1.tif");



In [42]:
pd.read_csv(fPath +'/{}'.format('gelPos.csv'),sep=',')[col]

,"x (px, z150_z225 Interface)","y (px, z150_z225 Interface)","z (px, z150_z225 Interface)"
0,683.238751,91.702693,3.029859
1,465.232563,165.483027,13.225994
2,795.208396,320.125112,10.697154
3,627.319752,80.580286,10.981324
4,727.869653,114.505114,13.284794
...,...,...,...
953,1818.814236,1716.332834,29.534083
954,1946.679483,1998.649421,8.508166
955,1716.212521,1755.003126,12.329902
956,1799.912922,1713.072853,16.580607


In [47]:
reload(flatField)
flatField.zStack2Mem( fPath+'/{}'.format('t0000_z150_z225.tif')).shape

(76, 2048, 2048)

# Conclusions (Feb 23) after looking at pyFiji for gel. 

1) I am missing particles in the gel, especially if particles are too close together.
2) Its also possible particles are missing for the same terracing effect that is just less obvious in dilute 